In [218]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

df = pd.read_csv('./survey.csv')

df.head()

replacement_ans = {
    "Agree": 1,
    "Unsure": 0,
    "Disagree": -1,    
}

replacement_conf = {
    "7-Extremely confident": 7,
    "6-Very confident": 6,
    "5-Somewhat confident": 5,
    "4-Neutral": 4,
    "3-Somewhat lacking confidence": 3,
    "2-Very lacking confidence": 2,
    "1-Extremely lacking confidence": 1,
}

condition = [
    "control",
    "ai-edited-image",
    "ai-edited-video",
    "ai-edited-image-video",
]

induced = [1,2,3,4,9,10,11,12,17,18,19,20]
non_induced_imgs = [5,6,7,8,13,14,15,16,21,22,23,24]

correct_is_true = [6,7,9,10,14,18,21,23]
correct_is_false = [1,2,3,4,5,8,11,12,13,15,16,17,19,20,22,24]

# get filtered columns
filtered_cols = [[f"{i}_false_memories_ans", f"{i}_false_memories_conf"] for i in [*range(1,25)]]
filtered_cols = [i for sublist in filtered_cols for i in sublist]

# remove headers and replace values
head = df.head(1)
df = df[2:][['condition', *filtered_cols]]
df.replace(replacement_ans, inplace=True)
df.replace(replacement_conf, inplace=True)

result = df.copy()
result = result[['condition'] + [f"{i}_false_memories_ans" for i in [*range(1,25)]]]

for i in [*range(1,25)]:
    multiplier = 1 if i in correct_is_true else -1
    result[f"{i}_false_memories_ans"] *= multiplier
    
# result -> 1 = true memory / 0 = unsure / -1 = false memory
result

/var/folders/bw/64y3yr9n6v50fgm86wtx236c0000gn/T/ipykernel_38742/3059041872.py:45: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(replacement_ans, inplace=True)
/var/folders/bw/64y3yr9n6v50fgm86wtx236c0000gn/T/ipykernel_38742/3059041872.py:46: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(replacement_conf, inplace=True)


,condition,1_false_memories_ans,2_false_memories_ans,3_false_memories_ans,4_false_memories_ans,5_false_memories_ans,6_false_memories_ans,7_false_memories_ans,8_false_memories_ans,9_false_memories_ans,...,15_false_memories_ans,16_false_memories_ans,17_false_memories_ans,18_false_memories_ans,19_false_memories_ans,20_false_memories_ans,21_false_memories_ans,22_false_memories_ans,23_false_memories_ans,24_false_memories_ans
2,control,1,0,-1,-1,1,1,1,0,1,...,-1,0,1,1,1,0,1,1,1,1
3,ai-edited-video,-1,1,-1,1,1,0,-1,1,1,...,0,0,1,1,1,1,1,0,-1,1
4,ai-edited-video,1,0,1,0,1,1,0,0,1,...,1,1,1,1,1,1,1,1,0,1
5,ai-edited-image-video,0,1,-1,0,0,-1,1,1,0,...,-1,0,0,1,-1,0,0,0,1,1
6,ai-edited-image,1,1,1,1,1,1,1,1,1,...,1,0,1,1,1,1,1,1,1,1
7,ai-edited-image,1,1,-1,0,1,-1,1,1,1,...,1,0,1,1,0,1,1,1,-1,1
8,ai-edited-image,0,0,-1,0,1,1,0,1,1,...,1,0,1,1,-1,-1,-1,1,-1,1
9,control,-1,-1,-1,-1,-1,1,1,-1,0,...,-1,1,1,-1,1,-1,-1,1,0,1
10,control,1,-1,1,-1,-1,0,-1,1,-1,...,1,1,1,-1,1,1,1,1,1,-1
11,ai-edited-image-video,1,1,-1,-1,1,1,1,1,-1,...,-1,1,-1,1,1,0,1,1,-1,1


In [217]:
def summarize(cat, df):
    # get sum
    df['overall'] = df[[f"{i}_false_memories_ans" for i in [*range(1,25)]]].sum(axis=1)
    df['induced'] = df[[f"{i}_false_memories_ans" for i in [*range(1,25)] if i in induced]].sum(axis=1)
    df['non induced'] = df[[f"{i}_false_memories_ans" for i in [*range(1,25) ] if i in non_induced_imgs]].sum(axis=1)

    # get percentages in % in two decimals
    df['overall %'] = (df['overall'] / 24) * 100
    df['induced %'] = (df['induced'] / 12) * 100
    df['non induced %'] = (df['non induced'] / 12) * 100

    # filter
    df = df[['condition', 'overall', 'overall %', 'induced', 'induced %', 'non induced', 'non induced %']]

    print(cat)
    display(df.groupby('condition').mean().round(2))
    
true_memories = result.copy().replace(-1,0)
false_memories = result.copy().replace(1,0).replace(-1,1)
unsure_memories = result.copy().replace(1,-1).replace(0,1).replace(-1,0)
summarize('Non False Memories', true_memories)
summarize('Unsure Memories', unsure_memories)
summarize('False Memories', false_memories)

Non False Memories


,overall,overall %,induced,induced %,non induced,non induced %
condition,,,,,,
ai-edited-image,16.0,66.67,7.33,61.11,8.67,72.22
ai-edited-image-video,13.0,54.17,5.00,41.67,8.00,66.67
ai-edited-video,15.5,64.58,8.00,66.67,7.50,62.50
control,14.0,58.33,6.33,52.78,7.67,63.89


Unsure Memories


,overall,overall %,induced,induced %,non induced,non induced %
condition,,,,,,
ai-edited-image,3.67,15.28,2.33,19.44,1.33,11.11
ai-edited-image-video,5.00,20.83,3.00,25.00,2.00,16.67
ai-edited-video,6.00,25.00,2.50,20.83,3.50,29.17
control,2.33,9.72,1.00,8.33,1.33,11.11


False Memories


,overall,overall %,induced,induced %,non induced,non induced %
condition,,,,,,
ai-edited-image,4.33,18.06,2.33,19.44,2.0,16.67
ai-edited-image-video,6.00,25.00,4.00,33.33,2.0,16.67
ai-edited-video,2.50,10.42,1.50,12.50,1.0,8.33
control,7.67,31.94,4.67,38.89,3.0,25.00


In [242]:
def confidence(cat, result, raw):
    print(cat)
    # display(result)
    # display(raw[['condition', *[f"{i}_false_memories_conf" for i in [*range(1,25)]]]])
    
    # get confidence: for each row {i}_confidence = result[{i}_false_memories_ans] * raw[{i}_false_memories_conf]
    for i in [*range(1,25)]:
        result[f"{i}_conf"] = result[f"{i}_false_memories_ans"] * raw[f"{i}_false_memories_conf"]
    
    result['overall'] = result[[f"{i}_conf" for i in [*range(1,25)]]].sum(axis=1) / 14
    result['induced'] = result[[f"{i}_conf" for i in [*range(1,25)] if i in induced]].sum(axis=1) / 7
    result['non induced'] = result[[f"{i}_conf" for i in [*range(1,25)] if i in non_induced_imgs]].sum(axis=1) / 7

    result = result[['condition', 'overall', 'induced', 'non induced']]
    display(result.groupby('condition').mean().round(2))
        
true_memories = result.copy().replace(-1,0)
false_memories = result.copy().replace(1,0).replace(-1,1)

confidence("Non False Memories", true_memories, df)
confidence("False Memories", false_memories, df)

Non False Memories


,overall,induced,non induced
condition,,,
ai-edited-image,5.86,5.43,6.29
ai-edited-image-video,5.14,4.07,6.21
ai-edited-video,6.46,6.57,6.36
control,5.12,4.76,5.48


False Memories


,overall,induced,non induced
condition,,,
ai-edited-image,1.43,1.76,1.10
ai-edited-image-video,2.29,3.21,1.36
ai-edited-video,0.96,1.21,0.71
control,2.60,3.14,2.05


In [256]:
score = pd.DataFrame()

# for each row in df, multiply answer by confidence
for i in [*range(1,25)]:
    multiplier = -1 if i in correct_is_false else 1
    score['condition'] = df['condition']
    score[f"{i}_false_memories_score"] = multiplier * df[f"{i}_false_memories_ans"] * df[f"{i}_false_memories_conf"]
    score[f"{i}_false_memories_no"] = multiplier * df[f"{i}_false_memories_ans"]

# sum scores    
score['overall'] = score[[f"{i}_false_memories_score" for i in [*range(1,25)]]].sum(axis=1)
score['induced'] = score[[f"{i}_false_memories_no" for i in [*range(1,25)] if i in induced]].sum(axis=1)
score['weighted induced'] = score[[f"{i}_false_memories_score" for i in [*range(1,25)] if i in induced]].sum(axis=1)
score['non induced'] = score[[f"{i}_false_memories_no" for i in [*range(1,25)] if i in non_induced_imgs]].sum(axis=1)
score['weighted non induced'] = score[[f"{i}_false_memories_score" for i in [*range(1,25)] if i in non_induced_imgs]].sum(axis=1)

cond = ['condition', 'overall', 'induced', 'weighted induced', 'non induced', 'weighted non induced']
# score = score[cond + [f"{i}_false_memories_score" for i in [*range(1,25)]]]
score = score[cond]

# group by condition and average
print(f"score: 1 -> correct (true memory), 0 -> unsure, -1 if wrong (false memory)")
print("weight score: score {1,0,-1} * confidence [1,7] (how strong the memory is)")
print("higher score means less false memory")
score.groupby('condition').mean().round(2)


score: 1 -> correct (true memory), 0 -> unsure, -1 if wrong (false memory)
weight score: score {1,0,-1} * confidence [1,7] (how strong the memory is)
higher score means less false memory


,overall,induced,weighted induced,non induced,weighted non induced
condition,,,,,
ai-edited-image,62.00,5.00,25.67,6.67,36.33
ai-edited-image-video,40.00,1.00,6.00,6.00,34.00
ai-edited-video,77.00,6.50,37.50,6.50,39.50
control,35.33,1.67,11.33,4.67,24.00
